In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold
from sklearn.linear_model import LinearRegression,LogisticRegression, LassoCV, RidgeCV, ElasticNetCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, RandomizedSearchCV
import itertools as it
import time as time
from catboost import CatBoostRegressor
import xgboost as xgb
from sklearn.ensemble import StackingRegressor, VotingRegressor
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV, ElasticNetCV
from pyearth import Earth

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [4]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)

# fit and transform the data
test_imputed = pd.DataFrame(imputer.fit_transform(test), columns=test.columns)
train_imputed = pd.DataFrame(imputer.fit_transform(train), columns=train.columns)

corr_matrix = train_imputed.corr().abs()

corr_y = train_imputed.corrwith(train_imputed['y'])
train_clean = train_imputed.drop(corr_y[corr_y.isna()].index.tolist(), axis = 1)
test_clean = test_imputed.drop(corr_y[corr_y.isna()].index.tolist(), axis = 1)

In [5]:
X = train_clean.drop(['id','y'], axis = 1)
y = train_clean.y
X_test = test_clean.drop('id', axis = 1)

In [6]:
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)
X_test_scaled = scaler.transform(X_test)

### MARS Feature Selection

In [10]:
mars = Earth(max_terms = 500, max_degree = 1, feature_importance_type = 'rss')
mars.fit(X_scaled, y)

KeyboardInterrupt: 

In [ ]:
predictor = pd.Series(X.columns, name = 'predictor')
rel_importance = pd.Series(mars.feature_importances_, name = 'relative importance')
importance = pd.concat([predictor, rel_importance], axis = 1).sort_values(by = 'relative importance', ascending = False)
mars_pred = list(importance['predictor'].loc[importance['relative importance']])

In [9]:
catboost = CatBoostRegressor(verbose = False).fit(X_scaled, np.log(y))

KeyboardInterrupt: 